# Fase de Pick

## Random Forest

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


#name             NPC hero name
#hero_id          Hero ID
#localized_name   In-game hero name

heroes= pd.read_csv("hero_names.csv")
heroes.head()


,name,hero_id,localized_name
0,npc_dota_hero_antimage,1,Anti-Mage
1,npc_dota_hero_axe,2,Axe
2,npc_dota_hero_bane,3,Bane
3,npc_dota_hero_bloodseeker,4,Bloodseeker
4,npc_dota_hero_crystal_maiden,5,Crystal Maiden


In [2]:
jugadores_test=pd.read_csv("test_player.csv")
jugadores_test.head()

,match_id,account_id,hero_id,player_slot
0,50000,117784,96,0
1,50000,158361,84,1
2,50000,158362,46,2
3,50000,137970,85,3
4,50000,1090,39,4


In [3]:
jugador_heroe = pd.merge(jugadores_test,heroes, on = 'hero_id',how = 'left')
jugador_heroe.set_index(['match_id','player_slot'],inplace = True)
jugador_heroe.head(10)

account_id  hero_id                            name  \
match_id player_slot                                                        
50000    0                117784       96           npc_dota_hero_centaur   
         1                158361       84         npc_dota_hero_ogre_magi   
         2                158362       46  npc_dota_hero_templar_assassin   
         3                137970       85           npc_dota_hero_undying   
         4                  1090       39       npc_dota_hero_queenofpain   
         128                2391        9            npc_dota_hero_mirana   
         129                2393       75          npc_dota_hero_silencer   
         130                2394      106      npc_dota_hero_ember_spirit   
         131               36737       74           npc_dota_hero_invoker   
         132                2392       62     npc_dota_hero_bounty_hunter   

                         localized_name  
match_id player_slot                     
50000    0            Centaur Warrunner  
         1                    Ogre Magi  
         2             Templar Assassin  
         3                      Undying  
         4                Queen of Pain  
         128                     Mirana  
         129                   Silencer  
         130               Ember Spirit  
         131                    Invoker  
         132              Bounty Hunter

In [4]:
jugadores_match=jugador_heroe['hero_id'].unstack()
jugadores_match.head()

player_slot,0,1,2,3,4,128,129,130,131,132
match_id,,,,,,,,,,
50000,96,84,46,85,39,9,75,106,74,62
50001,44,10,57,2,106,58,61,21,18,14
50002,74,7,42,99,88,69,8,25,26,79
50003,44,15,110,56,94,2,101,32,7,72
50004,98,26,73,51,46,2,106,50,65,21


In [6]:
partido_equipo=pd.read_csv("test_labels.csv")
partido_equipo.head()

,match_id,radiant_win
0,50000,0
1,50001,1
2,50002,0
3,50003,1
4,50004,0


In [7]:
test_pick=pd.merge(partido_equipo,jugadores_match,left_index = True,right_index = True)
test_pick.head()

,match_id,radiant_win,0,1,2,3,4,128,129,130,131,132
50000,100000,0,96,84,46,85,39,9,75,106,74,62
50001,100001,1,44,10,57,2,106,58,61,21,18,14
50002,100002,0,74,7,42,99,88,69,8,25,26,79
50003,100003,0,44,15,110,56,94,2,101,32,7,72
50004,100004,0,98,26,73,51,46,2,106,50,65,21


In [8]:
test_pick=pd.merge(partido_equipo,jugadores_match,left_index = True,right_index = True)
test_pick.head()

,match_id,radiant_win,0,1,2,3,4,128,129,130,131,132
50000,100000,0,96,84,46,85,39,9,75,106,74,62
50001,100001,1,44,10,57,2,106,58,61,21,18,14
50002,100002,0,74,7,42,99,88,69,8,25,26,79
50003,100003,0,44,15,110,56,94,2,101,32,7,72
50004,100004,0,98,26,73,51,46,2,106,50,65,21


In [9]:
test_pick.isna().sum()

match_id       0
radiant_win    0
0              0
1              0
2              0
3              0
4              0
128            0
129            0
130            0
131            0
132            0
dtype: int64

In [10]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

from sklearn.model_selection import cross_val_score

In [11]:
X=test_pick[test_pick.columns.difference(["match_id","radiant_win"])]
y=test_pick["radiant_win"]
y.head()
X.head()

,0,1,2,3,4,128,129,130,131,132
50000,96,84,46,85,39,9,75,106,74,62
50001,44,10,57,2,106,58,61,21,18,14
50002,74,7,42,99,88,69,8,25,26,79
50003,44,15,110,56,94,2,101,32,7,72
50004,98,26,73,51,46,2,106,50,65,21


In [12]:
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X,y,test_size=0.2)

print(X_entrenamiento)

       0    1    2    3    4    128  129  130  131  132
79165   34  106  100   64   69   42   51   28   74   50
97535   93   22   14   46   69   11   61   68   73   86
95787   28   50   54   94   74   19   25   22   68  106
84899    1    7   73  106    9   81  104    8   22   57
91166   11   68   21   23   67   93    8   57    5    3
77990    4   21   31   71  100   10   41   26   30   35
65193   21  112   28   46   50   14   63   44   39   62
53382   31  100   62   69  106  112  102   49   36   25
50919   86    7  104    8   28    2    1   39   62  107
94106  106   55   87   42  102   11   86   69   51   25
79245   71   30   64   72   47   78   68  106   11    7
70773   26   25   73   62   46  112  104    8   22   35
56787   69   54  104    9   82   20   73   93    7   36
56702  110   74   12   60   29   86   51    7   25   72
62677   34   42   68    9   31   72   51  112   76   60
82455    8    2   17   71   57   55   61  106   27   85
72939   28  106   40   74   86   26  100    8   

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
RFC=RandomForestClassifier(n_estimators=10).fit(X_entrenamiento,y_entrenamiento)

predicciones_test_RFC=RFC.predict(X_prueba)

matriz_confusion_RFC = metrics.confusion_matrix(y_prueba, predicciones_test_RFC)
TN_LR = matriz_confusion_RFC[0,0]
FN_LR = matriz_confusion_RFC[1,0]
FP_LR = matriz_confusion_RFC[0,1]
TP_LR = matriz_confusion_RFC[1,1]

print ('              +-----------------+')
print ('              |   Predicción    |')
print ('              +-----------------+')
print ('              |    +   |    -   |')
print ('+-------+-----+--------+--------+')
print ('| Valor |  +  |   %d |  %d     |'   % (TP_LR, FN_LR) )
print ('| real  +-----+--------+--------+')
print ('|       |  -  | %d   | %d      |'    % (FP_LR, TN_LR) )
print ('+-------+-----+--------+--------+')
print ()
#print ( 'Exactitud    : ', (TP_LR+TN_LR)/(TP_LR+FN_LR+FP_LR+TN_LR) )
print('Exactitud     :',metrics.accuracy_score(y_prueba, predicciones_test_RFC) )
print('Precisión     :',metrics.precision_score(y_prueba, predicciones_test_RFC)) 
print('Exhaustividad :',metrics.recall_score(y_prueba, predicciones_test_RFC))
print('F1-Score      :',metrics.f1_score(y_prueba, predicciones_test_RFC))

              +-----------------+
              |   Predicción    |
              +-----------------+
              |    +   |    -   |
+-------+-----+--------+--------+
| Valor |  +  |   2335 |  2862     |
| real  +-----+--------+--------+
|       |  -  | 2047   | 2756      |
+-------+-----+--------+--------+

Exactitud     : 0.5091
Precisión     : 0.5328617069831127
Exhaustividad : 0.4492976717336925
F1-Score      : 0.4875247938198142


In [22]:
import tensorflow as tf
from tensorflow import keras

print()

79165    0
97535    1
95787    1
84899    1
91166    1
77990    1
65193    1
53382    1
50919    1
94106    0
79245    0
70773    1
56787    0
56702    1
62677    1
82455    0
72939    0
76590    1
59584    0
98944    0
93211    1
53935    0
94078    0
76489    1
92057    1
59282    1
98149    1
89155    1
88309    1
66321    1
        ..
81413    0
78888    0
83031    0
78454    1
73591    1
78819    1
54409    1
75000    0
69398    1
95600    1
81087    0
91217    0
89994    0
65739    1
69902    1
68109    1
58062    0
79417    0
60345    0
70458    0
91764    0
62394    0
75111    1
62348    1
95081    1
64921    0
66997    1
85367    1
63444    1
88444    1
Name: radiant_win, Length: 40000, dtype: int64


In [24]:
X = numpy.zeros([157, 128])
Y = numpy.zeros([157], dtype=numpy.int32)
example_id = numpy.array(['%d' % i for i in range(len(Y))])

x_column_name = 'x'
example_id_column_name = 'example_id'

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={x_column_name: X, example_id_column_name: example_id},
    y=Y,
    num_epochs=None,
    shuffle=True)

svm = tf.contrib.learn.SVM(
    example_id_column=example_id_column_name,
    feature_columns=(tf.contrib.layers.real_valued_column(
        column_name=x_column_name, dimension=128),),
    l2_regularization=0.1)

svm.fit(input_fn=train_input_fn, steps=10)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_worker_replicas': 0, '_model_dir': '/tmp/tmp5jbaaies', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe593f10ba8>, '_keep_checkpoint_every_n_hours': 10000, '_train_distribute': None, '_environment': 'local', '_save_summary_steps': 100, '_tf_random_seed': None, '_task_id': 0, '_session_config': None, '_device_fn': None, '_save_checkpoints_steps': None, '_is_chief': True, '_log_step_count_steps': 100, '_eval_distribute': None, '_protocol': None, '_evaluation_master': '', '_save_checkpoints_secs': 600, '_master': '', '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_task_type': None}
